### Important Note:
PyCharm's Jupyter is not supported when using WSL, so need to use regular Windows interpreter

In [6]:
import time
import numpy as np
import cv2

import final_project.utils as u

Measure the time it takes to detect and match within-frame
Also, extract left descriptors of inliers, to use when measuring between-frame matchings

In [13]:
def measure_within_frame(detector, matcher):
    num_frames = 3450
    detect_times = np.zeros((num_frames,))

    regular_match_times = np.zeros((num_frames,))
    regular_match_counts = np.zeros((num_frames,))
    regular_match_inlier_descriptors = []

    knn_match_times = np.zeros((num_frames,))
    knn_match_counts = np.zeros((num_frames,))
    knn_match_inlier_descriptors = []

    for idx in range(num_frames):
        img_l, img_r = u.read_images(idx)

        start = time.time()
        kps_left, desc_left = detector.detectAndCompute(img_l, None)
        kps_right, desc_right = detector.detectAndCompute(img_r, None)
        detect_times[idx] = time.time() - start

        start = time.time()
        regular_matches = matcher.match(desc_left, desc_right)
        inlier_descriptors = []
        for m in regular_matches:
            kpl, kpr = kps_left[m.queryIdx], kps_left[m.trainIdx]
            yl, yr = kpl.pt[1], kpr.pt[1]
            if abs(yl - yr) <= 1:
                inlier_descriptors.append(desc_left[m.queryIdx])
        regular_match_times[idx] = time.time() - start
        regular_match_counts[idx] = len(regular_matches)
        regular_match_inlier_descriptors.append(np.array(inlier_descriptors))

        start = time.time()
        knn_matches = matcher.knnMatch(desc_left, desc_right, 2)
        good_matches = [first for (first, second) in knn_matches if first.distance / second.distance <= 0.75]
        inlier_descriptors = []
        for m in good_matches:
            kpl, kpr = kps_left[m.queryIdx], kps_left[m.trainIdx]
            yl, yr = kpl.pt[1], kpr.pt[1]
            if abs(yl - yr) <= 1:
                inlier_descriptors.append(desc_left[m.queryIdx])
        knn_match_times[idx] = time.time() - start
        knn_match_counts[idx] = len(knn_matches)
        knn_match_inlier_descriptors.append(np.array(inlier_descriptors))
        break
    return detect_times, regular_match_times, regular_match_counts, regular_match_inlier_descriptors, knn_match_times, knn_match_counts, knn_match_inlier_descriptors

Measure matching between consecutive frames, based on descriptors from within-frames matching

In [8]:
def measure_between_frame(matcher, left_descriptors):
    num_frames = 3450

    regular_match_times = np.zeros((num_frames,))
    regular_match_counts = np.zeros((num_frames,))

    knn_match_times = np.zeros((num_frames,))
    knn_match_counts = np.zeros((num_frames,))

    for idx in range(1, num_frames):
        prev_desc = left_descriptors[idx-1]
        curr_desc = left_descriptors[idx]

        start = time.time()
        regular_matches = matcher.match(prev_desc, curr_desc)
        regular_match_times[idx] = time.time() - start
        regular_match_counts[idx] = len(regular_matches)

        start = time.time()
        knn_matches = matcher.knnMatch(prev_desc, curr_desc, 2)
        knn_match_times[idx] = time.time() - start
        knn_match_counts[idx] = len(knn_matches)
    return regular_match_times, knn_match_counts, knn_match_times, knn_match_counts

## Start Measuring

### Pair #1
Detector: ORB
Matcher: BF (no Cross-Check)

In [15]:
det_type = "ORB"
mtchr_type = "BF (no Cross-Check)"

print(f"Detector:\t{det_type}")
print(f"Matcher:\t{mtchr_type}")

det = cv2.ORB_create()
mtchr = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)

within_frame_results = measure_within_frame(det, mtchr)

detect_times = within_frame_results[0]
regular_match_times = within_frame_results[1]
regular_match_counts = within_frame_results[2]
regular_match_inlier_descriptors = within_frame_results[3]
regular_match_inlier_percents = np.array([len(regular_match_inlier_descriptors[i]) / regular_match_counts[i] for i in range(len(regular_match_inlier_descriptors))])

knn_match_times = within_frame_results[4]
knn_match_counts = within_frame_results[5]
knn_match_inlier_descriptors = within_frame_results[6]
knn_match_inlier_percents = np.array([len(knn_match_inlier_descriptors[i]) / knn_match_counts[i] for i in range(len(knn_match_inlier_descriptors))])

print("\tDetection:")
print(f"\tTimes:\t\tMean:\t{detect_times.mean():.2f}\t\tMin:\t{detect_times.min():.2f}\t\tMax:\t{detect_times.max():.2f}")

print("\tRegular Matching Within:")
print(f"\tTimes:\t\tMean:\t{regular_match_times.mean():.2f}\t\tMin:\t{regular_match_times.min():.2f}\t\tMax:\t{regular_match_times.max():.2f}")
print(f"\tCounts:\t\tMean:\t{regular_match_counts.mean():.2f}\t\tMin:\t{regular_match_counts.min():.2f}\t\tMax:\t{regular_match_counts.max():.2f}")
print(f"\tInliers:\t\tMean:\t{regular_match_inlier_percents.mean():.2f}\t\tMin:\t{regular_match_inlier_percents.min():.2f}\t\tMax:\t{regular_match_inlier_percents.max():.2f}")

print("\tKNN Matching Within:")
print(f"\tTimes:\t\tMean:\t{knn_match_times.mean():.2f}\t\tMin:\t{knn_match_times.min():.2f}\t\tMax:\t{knn_match_times.max():.2f}")
print(f"\tCounts:\t\tMean:\t{knn_match_counts.mean():.2f}\t\tMin:\t{knn_match_counts.min():.2f}\t\tMax:\t{knn_match_counts.max():.2f}")
print(f"\tInliers:\t\tMean:\t{knn_match_inlier_percents.mean():.2f}\t\tMin:\t{knn_match_inlier_percents.min():.2f}\t\tMax:\t{knn_match_inlier_percents.max():.2f}")

Detector:	ORB
Matcher:	BF (no Cross-Check)
	Detection:
	Times:		Mean:	0.00		Min:	0.00		Max:	0.02
	Regular Matching Within:
	Times:		Mean:	0.00		Min:	0.00		Max:	0.00
	Counts:		Mean:	0.14		Min:	0.00		Max:	500.00
	Inliers:		Mean:	0.02		Min:	0.02		Max:	0.02
	KNN Matching Within:
	Times:		Mean:	0.00		Min:	0.00		Max:	0.00
	Counts:		Mean:	0.14		Min:	0.00		Max:	500.00
	Inliers:		Mean:	0.01		Min:	0.01		Max:	0.01


In [ ]:
orb_detector = cv2.ORB_create()
sift_detector = cv2.SIFT_create()

bf_matcher_no_cs = cv2.BFMatcher(norm=cv2.NORM_L2, cross_check=False)
bf_matcher_with_cs = cv2.BFMatcher(norm=cv2.NORM_L2, cross_check=True)
flann_matcher = cv2.FlannBasedMatcher(indexParams=dict(algorithm=0, trees=5), searchParams=dict(checks=50))